# Create a Wide-Area Ethernet (Layer 2) Network

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it.  


## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated WAN Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If the interfaces in the list are located on two sites, the network will automatically create a wide-area layer 2 circuit.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
slice_name = 'MySlice120'
[site1,site2]  = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('config')
net1.add_interface(iface1)
iface1.set_ip_addr(IPv4Address("192.168.1.1"))


# Node2
node2 = slice.add_node(name=node2_name, site=site2)
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)
iface2.set_ip_addr(IPv4Address("192.168.1.2"))


#Submit Slice Request
slice.submit()

## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()